<h1 style="font-size:150px">FleetParser</h1>

## Importing necessary libraries

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import grequests
import requests as rq

import os
import tqdm
import numpy as np
from datetime import datetime
from pathlib import Path
from typing import List, Tuple, Iterable, Union
import transliterate
import itertools
from difflib import SequenceMatcher
from heapq import nlargest as _nlargest
import shutil
import lxml.html
import warnings
warnings.simplefilter("ignore", category=UserWarning)

import time

### If you start parser first time, install the following library:

In [ ]:
# !pip install transliterate

# Config

## Headers to avoid spam block

In [ ]:
# headers and cookies to work with
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
          "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
          }

## Parameters of matching

In [ ]:
icaos_aircraft = {"iata": True, "icao": True, "wiki1": True, 
         "wiki2": False, "wiki3": False, "fan": True, 
         "mapper": True, "military": False, 'flug_iata': False, 
         'flug_icao': True, 'av': True
        }

iatas_aircraft = {"iata": True, "icao": False, "wiki1": True, 
         "wiki2": False, "wiki3": False, "fan": False, 
         "mapper": True, "military": False, 'flug_iata': True, 
         'flug_icao': False, 'av': True
        }

icaos_airlines = {
                  "code":True,
                  "av_airlines":True,
                  "flug_airlines":True,
                  "aircodes":True
                }

iatas_airlines = {
                  "code":True,
                  "av_airlines":True,
                  "flug_airlines":True,
                  "aircodes":True
                 }

database_url = {"iata": 'http://wiki.aviabit.ru/doku.php?id=pub:ssim._apendix_a', 
                "icao": 'https://www.icao.int/publications/DOC8643/Pages/Search.aspx', 
                "wiki1": 'https://en.wikipedia.org/wiki/List_of_aircraft_type_designators',
                "wiki2": 'https://en.wikipedia.org/wiki/List_of_civil_aircraft', 
                "wiki3": 'https://en.wikipedia.org/wiki/List_of_aircraft_by_date_and_usage_category', 
                "fan": 'https://www.aviationfanatic.com/ent_list.php?ent=4', 
                "mapper": 'https://www.greatcirclemapper.net/en/aircrafts.html', 
                "military": 'https://www.militaryfactory.com/aircraft/indexMAIN.php', 
                'flug_iata': 'http://www.flugzeuginfo.net/table_accodes_iata_en.php', 
                'flug_icao': 'http://www.flugzeuginfo.net/table_accodes_en.php', 
                'av': 'https://www.avcodes.co.uk/acrtypes.asp',
                "code": 'https://en.wikipedia.org/wiki/List_of_airline_codes',
                "av_airlines":'https://www.avcodes.co.uk/airlcodesearch.asp',
                "flug_airlines":'http://www.flugzeuginfo.net/table_airlinecodes_airline_en.php',
                "aircodes":'https://airlinecodes.info/icao'
        }



icao_aircraft_lst = [key for key, value in icaos_aircraft.items() if value and key != 'av']
icao_airlines_lst = [key for key, value in icaos_airlines.items() if value]

skip_list = [
#     'wiki2', 'wiki3',  
    'military',
    'iata',
#     'wiki2', 
    'av'
]


## Some exceptions that needs to be separately indicated

In [ ]:
change_values_of_aircrafts = ['Airbus a319-100','Airbus a318-100','Airbus a319-100lr',
                    'Airbus a320-200','Airbus a320-231','Airbus a321-100',
                    'Airbus a321-200','Airbus a321-200p2f','Airbus a321-p2f',
                    'Ан', 'Ми']

# Functions

## Gathering of initial parse objects

In [ ]:
def wiki_urls(url, table_num, col_num, return_all=False, spec_xpath=None):
    """
    Parse wiki table to get column values and their's hyperlinks
    
    Parameters
    ----------
    url: str
        http-like string url to wiki page
    
    table_num: int
        number of table on page
        
    col_num: int
        number of column to get values from
        
    return_all: bool, default False
        flag, specifying particular page with different xpath
    
    spec_xpath: str, default None
        specify xpath to the table values that needs to be gathered
    
    """
    html = rq.get(url)
    doc = lxml.html.fromstring(html.content)
    
    if return_all:
        xpth = '//*[@id="mw-content-text"]/div[1]/ul[*]/li[*]/a'
        names = doc.xpath(f'{xpth}/text()')
        urls = doc.xpath(f'{xpth}/@href')
        return names, ['https://en.wikipedia.org'+url for url in urls]
    
    if spec_xpath:
        names = doc.xpath(f'{spec_xpath}/text()') 
        urls = doc.xpath(f'{spec_xpath}/@href')
        return names, ['https://en.wikipedia.org'+url for url in urls]
    
    xpth = f'//*[@id="mw-content-text"]/div[1]/table[{table_num}]/tbody/tr[*]/td[{col_num}]/a'
    names = doc.xpath(f'{xpth}/text()')
    urls = doc.xpath(f'{xpth}/@href')
    
    return names, ['https://en.wikipedia.org'+url for url in urls]

In [ ]:
def get_list_companies(source='pulkovo', mode='urls', headers=headers) -> List[str]:
    """
    Parse list of companies cooperating with 
    Pulkovo from the airport website.
    Or gather companies from different 
    wiki page through specifying source parameter

    Parameters
    ----------
    source: {'pulkovo', 'us', 'europe', 'cac', 'africa'
            , 'asia', 'northamerica', 'oceania', 'southamerica',
            'all', 'test', 'national, 'pulkovo_preloaded'}, default 'pulkovo'
        Available sources to get info about airline companies. 
        There are number of regions available.
        NOTE! 'cac' stands for "Central america and the Caribbean".
        NOTE! You can specify only 'names' if the source=='pulkovo'.
        NOTE! You can use source=='pulkovo_preloaded', only if you have appropriate dataset.
        
    mode: {'names', 'urls'}, default 'urls'
        What data to return
        NOTE! You can specify only 'names' if the source=='pulkovo'
    
    headers: dict
        Headers to use for the url request 
        (necessary for avoiding spam-block)

    
    """
    if source == 'pulkovo':
        
        url='https://pulkovoairport.ru/passengers/destinations/airlines/#'
        
        response = rq.get(url, headers=headers)
        to_parse = response.text
        soup = BeautifulSoup(to_parse, 'html.parser')
        companies = soup.find('ul', {'class': 'tm-air uk-accordion'}).find_all('a', {'class': 'uk-accordion-title'})
        names = []
        for company in companies:
            names.append(company.text)
    
    elif source == 'pulkovo_preloaded':
        plk = pd.read_excel('pulkovo_preloaded.xlsx')
        names, urls = plk.names.tolist(), plk.urls.tolist()
        
    elif source == 'us':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_airlines_of_the_United_States', table_num=3, col_num=1) #us
        
    elif source == 'europe':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_largest_airlines_in_Europe', table_num=1, col_num=2) #eu
        
    elif source == 'cac':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_largest_airlines_in_Central_America_and_the_Caribbean', table_num=2, col_num=2) #cac
    
    elif source == 'africa':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_largest_airlines_in_Africa', table_num=2, col_num=2) #africa
    
    elif source == 'asia':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_largest_airlines_in_Asia','',2, spec_xpath='//*[@id="mw-content-text"]/div[1]/div[1]/table/tbody/tr[*]/td[2]/a')
    
    elif source == 'northamerica':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_largest_airlines_in_North_America', table_num=2, col_num=1) #na
    
    elif source == 'oceania':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_largest_airlines_in_Oceania', table_num=1, col_num=1) #oceania
        
    elif source == 'southamerica':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_largest_airlines_in_South_America', table_num=1, col_num=2) #asia
    
    elif source == 'all':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_passenger_airlines',0,0,True)
        
    elif source == 'test':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/List_of_airline_codes', 1, 3)
        
    elif source == 'national':
        names, urls = wiki_urls('https://en.wikipedia.org/wiki/Flag_carrier', 1, 2)
        
    else:
        
        return names
        
    if mode == 'urls':
        print(f'Number of URLs to iterate through: {len(urls)}')
        return urls
    
    if mode == 'names':
        print(f'Number of NAMES to search and iterate through: {len(names)}')
        return names
    
    

## Algorithm for searching company's article URL by its name

In [ ]:
def process_query(company: str, mode='ru', append='+авиакомпания') -> Tuple[str]:
    """
    Construct the search query to look through wikipedia.org and 
    get the title and referer url of the first most apropriate result

    Parameters
    ----------
    company: str
        Name of company to use for constructing a search query
    
    mode: str
        Type of wikipedia library language to use for search

    append: str
        Keywords to use for constructing a search 
        query for better accuracy of received result.
        NOTE! use + in start of a string and as a separator 
        between words in append
    
    """

    company = company.replace('«','%22').replace('»','%22').replace('Group', '').replace('Holdings', '')
    search = '+'.join(company.split())
    query = f'https://{mode}.wikipedia.org/w/index.php?search={search}{append}&title=Special:Search&profile=advanced&fulltext=1&ns0=1'
    response = rq.get(query, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    href = soup.find('div', {"class":"mw-search-result-heading"}).find('a').get('href')
    title = soup.find('div', {"class":"mw-search-result-heading"}).find('a').get('title')
    
    return title, href

In [ ]:
def get_url(company: str) -> str:
    """
    Compare the result and try to get the 
    most valuable result with desired company 
    url to parse further

    Parameters
    ----------
    company: str
        Name of company to look through
    
    """
    try:
        ru_flag = False
        pat = re.compile(company, re.IGNORECASE)
        ru = re.compile("[а-яА-Я]")
        en = re.compile("[a-zA-Z]")
        ru_filter = filter(ru.match, company)
        en_filter = filter(en.match, company)
        if list(ru_filter):

            title, href = process_query(company)
            ru_flag = True

        else:

            title, href = process_query(company, mode='en', append='+airline')

        if not re.match(pat, title):

            if ru_flag:

                title, href = process_query(company, mode='ru', append='')

            else:

                title, href = process_query(company, mode='en', append='')

        if ru_flag:      
            link_ = f'https://ru.wikipedia.org{href}'
        else:
            link_ = f'https://en.wikipedia.org{href}'

        response_url = rq.get(link_, headers=headers)
        soup_url = BeautifulSoup(response_url.text, 'html.parser')
        link = soup_url.find('a', {'class':'interlanguage-link-target', 'lang':'en'})
        link = link.get('href') if link is not None else link_

        return link
    
    except:
        
        return None

# Processing and transformation of parsed DataFrame

In [ ]:
def get_company_name(response) -> str:
    """
    Get the full name of the company from the title of its wiki page

    Parameters
    ----------
    response: requests.Response
        response corresponding to particular wikipage

    """

    soup = BeautifulSoup(response.text[:15000], 'html.parser') \
        .find('h1' ,{"class" :"firstHeading mw-first-heading"}).text
    return soup

In [ ]:
def handle_multiindex(df):
    """
    Transform multiindex of dataframe to get pretty looking column names
    
    Parameters
    ----------
    df: pd.DataFrame
        Dataframe whose columns needs to be processed
    
    """
    def unique_list(l):
        ulist = []
        [ulist.append(x.strip('01234567890 []')) for x in l if x not in ulist]
        return ulist
    new_cols = []
    if df.columns.nlevels > 1:
        df.columns = df.columns.map('|'.join)
    else:
        df.columns = df.columns.map(str).map(''.join)
    for col in df.columns:
        splt = col.split('|')
        new_cols.append(' '.join(unique_list(splt)))
    
    return new_cols

In [ ]:
def transform_columns(df: Iterable) -> tuple:
    """
    Process columns of given DataFrame to get only necessary and apropriate columns. 

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe, whose columns should be processed
    
    
    """
    dct = {'Aircraft': [], 'In service': [], 'Orders': [], 'Total_Passengers': [], 'Notes': [], }
    match = ('Aircraft|Aircraft type|Type|Plane Name|Тип самолёта',
             'In service|Inservice|In operation|No. of aircraft|No. ofaircraft|In fleet|^Fleet$|InFleet|^Total$|Number in Operation|^Number$|^Count$|In-service|Fleet (TC/PLR)|TC list|Active|Эксплуатируется',
             'Order|Orders|Заказано',
             'Passengers|PassengerTotal|^Passenger|Passengers(Economy)|Passenger capacity|Seating Class|Seats',
             'Notes|Additional|Примечания')
    
    df.columns = handle_multiindex(df)
    
    for pat, pattern in enumerate(match):

        for num, col in enumerate(df.columns):

            if re.search(re.compile(pattern, re.IGNORECASE), str(col)):
                dct[list(dct.keys())[pat]] = dct.get(list(dct.keys())[pat], []) + [(num, col)]

    to_keep = []
    for key, values in dct.items():

        if len(values)>0:

            if key == 'Aircraft':
                to_keep.append(values[0][0])

            if key == 'In service':
                to_keep.append(values[0][0])

            if key == 'Orders':
                to_keep.append(values[0][0])

            if key == 'Total_Passengers':
                ref = False
                for val in values:
                    if 'Ref.' in val[1] or 'Refs' in val[1]:
                        ref = True
                if ref:
                    to_keep.append(values[-2][0])
                else:
                    to_keep.append(values[-1][0])    

            if key == 'Notes':
                to_keep.append(values[0][0])



    all_idx = [i for i in range(len(df.columns))]
    to_del = list(set(all_idx)-set(to_keep))
    cols = df.columns[to_keep]
    return cols, to_del, to_keep

In [ ]:
def get_desigs(response, cut=20000):
    """
    Try to get designator values (IATA, ICAO, Callsign) from response of parcticular wikipage
    
    Parameters
    ----------
    response: requests.Response
        Response from parcticular wikipage
    
    cut: int
        How much text to leave for getting the designators (influences the perfomance)
    
    
    """
    
    try:
        soup_nick = BeautifulSoup(response.text[:cut], 'html.parser').find('td', {'class': 'infobox-full-data'}).find_all('td', {'class':'nickname'})
        iata = re.sub(r"\[.*?\]", '', soup_nick[0].text.strip())
        icao = re.sub(r"\[.*?\]", '', soup_nick[1].text.strip())
        callsign = re.sub(r"\[.*?\]", '', soup_nick[2].text.strip())
        return iata, icao, callsign
    except:
        return np.nan, np.nan, np.nan

In [ ]:
def transform_table(df: pd.DataFrame, company: str, response) -> pd.DataFrame:
    """
    Transform given DataFrame table to keep only 
    necessary info, get the appropriate shape and 
    unify to a common table form  

    Parameters
    ----------
    df: pd.DataFrame
        DataFrame to handle with
    
    company: str
        Company name to use in table
    
    response: requests.Response
        Response from particular wikipage
    
    """

    cols, to_del, to_keep = transform_columns(df)

    df = df.iloc[:, to_keep].copy()

    df.columns = df.columns.map(''.join)

    df.columns = cols
    add_columns(df) 
    rename_df_cols(df)

    for col in df.columns:
        df[col] = df[col].replace(re.compile('\[.*?\]'),'').replace(re.compile('[0-9a]*/'), '')
    
    iata, icao, callsign = get_desigs(response)
    
    df['Company'] = company
    df['Wiki URL'] = response.url
    df['Airline IATA'] = iata
    df['Airline ICAO'] = icao
    df['Callsign'] = callsign
    
    try:
        df = df[(~df.Aircraft.str.lower().str.contains('total'))&(~df.Aircraft.str.lower().str.contains('всего'))&(~df.Aircraft.str.lower().str.contains('total:'))]
    except:
        pass

    return df

In [ ]:
def add_columns(df: pd.DataFrame) -> None:
    """
    Add missing columns to the given DataFrame 
    to get its shape to complete and apropriate form 

    Parameters
    ----------
    df: pd.DataFrame
        DataFrame to handle with

    """
    
    add = []
    match = ('Aircraft|Aircraft type|Type|Plane Name|Тип самолёта',
             'In service|Inservice|No. of aircraft|In fleet|^Fleet$|^Total$|InFleet|No. ofaircraft|In operation|In-service|Number in Operation|TC list|^Count$|Fleet (TC/PLR)|^Number$|Active|Эксплуатируется',
             'Order|Orders|Заказано',
             'Passengers|Passengers(Economy)|^Passenger|PassengerTotal|Passenger capacity|Seating Class|Seats',
             'Notes|Additional|Примечания')
    
    required = ['AirCraft', 'In service', 'Orders', 'Total_Passengers', 'Notes']
    for pat, pattern in enumerate(match):
        status = False
        
        for num, col in enumerate(df.columns):
            if re.search(re.compile(pattern, re.IGNORECASE), str(col)):
                status = True
        if not status:
            add.append(pat)
            
    for item in sorted(list(set(add))):
        length = len(df.columns)
        if item < length:
            df.insert(item, required[item], np.nan)
        else:
            df[required[item]] = np.nan

In [ ]:
def rename_df_cols(df: pd.DataFrame) -> None:
    """
    Rename the columns of given DataFrame.
    DataFrame should be of specific form

    Parameters
    ----------
    df: pd.DataFrame
        DataFrame whose columns to rename

    """
    
    cols = ['Aircraft','In service', 'Orders', 'Total_Passengers','Notes']
    df.columns = cols

# Parse engine implementation

In [ ]:
def extract_txt(response, headers: dict, id_value=re.compile('Флот|Fleet'), one_more_url=False) -> str:
    """
    Extract text containing information about aviacompany
    fleet from the "Fleet" subtitle of wiki page

    Parameters
    ----------
    response: requests.Response
        Response from particular wikipage

    headers: dict
        headers to post on the wiki server

    id_value: re.compile(..) object, default re.compile('Флот|Fleet')
        id_value of "Fleet" subtitle to find desired text
        
    one_more_url: bool, default False
        Specifies if table might be on a different page

    """
    if one_more_url:
        response = rq.get(response, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    h2_tags = soup.find_all('h2')

    for index, h2_tag in enumerate(h2_tags):
        span_tag = h2_tag.find('span', {'id': id_value})
        if span_tag:
            break

    start_index = index
    finish_index = start_index + 1
    start_pattern = h2_tags[start_index].find('span')
    finish_pattern = h2_tags[finish_index].find('span')
    start = re.search(str(start_pattern), response.text).span()[0]
    finish = re.search(str(finish_pattern), response.text).span()[1]
    text = response.text[start:finish]

    return text

In [ ]:
def extract_table(text: str, url, header=None, match='.+', one_more_url=False) -> pd.DataFrame:
    """
    Extract pd.DataFrame table from the given html-like string

    Parameters
    ----------
    text: str
        HTML string to get table from,
        consisting of <table>, <th>, <td> tags

    response (url): requests.Response
        Response from particular wikipage

    header: int, default None
        row index to specify header row and
        start table

    match: str, or re.compile object, default '.+'
        pattern to specify table search
        
    one_more_url: bool, default False
        Specifies if table might be on a different page

    """
    if not one_more_url:
        url = url.url

    if 'ru.wikipedia.org' in url:
        df = pd.read_html(text, header=0, match=match)
    else:
        df = pd.read_html(text, header=header, match=match)
    return df

In [ ]:
def get_no_table(response, company: str, headers: dict) -> tuple:
    """
    Extract no-table elements with information
    about fleet and convert it to specific form DataFrame,
    containing complete information about some
    company fleet from its Wiki page

    Parameters
    ----------
    response: requests.Response
        Response from specific wikipage

    company: str
        name of the company to use for
        forming DataFrame

    headers: dict
        headers to post on the wiki server

    """

    try:

        info = []

        df = pd.DataFrame()

        text = extract_txt(response, headers)

        soup = BeautifulSoup(text, 'html.parser')

        if soup.find('li') and soup.find('ul'):

            soup_ul = BeautifulSoup(text, 'html.parser').find_all('ul')

            for ul in soup_ul:
                for li in ul.find_all('li'):
                    info.append(li.text)
        
        # ------------------------------------
            crafts = []
            pat = r'^([0-9])*| – ([0-9]*)'
            pat2 = r"\(.*?\)"
            notes = ['' for craft in info]
            for ind in range(len(info)):
                notes[ind] = re.search(pat2, info[ind]).group(0) if re.search(pat2, info[ind]) else ''
                if not notes[ind]:
                    notes[ind] = ', '.join(info[ind].rsplit(',')[1:]) if len(info[ind].rsplit(',', 1)) > 1 else ''
                if not notes[ind]:
                    notes[ind] = info[ind].rsplit(';', 1)[1] if len(info[ind].rsplit(',', 1)) > 1 else ''
                if not notes[ind]:
                    notes[ind] = info[ind].rsplit(':', 1)[1] if len(info[ind].rsplit(',', 1)) > 1 else ''
                if notes[ind]:
                    add_pat = notes[ind]
                else:
                    add_pat = 'nopattern'
                pat3 = re.compile(f'{pat2}|{add_pat}')
                info[ind] = re.sub(pat3, '', info[ind]).strip(' , ;')
                if ';' in info[ind]:
                    info[ind] = info[ind].rsplit(';',1)[0].strip()
            for craft in info:
                mod = re.split(pat, craft)
                mod1 = [craft.lstrip(' x×–-\n').rstrip('-\n') for craft in mod if craft]
                mod2 = sorted(mod1)
                crafts.append(mod2)
            [craft.insert(0, "1") for craft in crafts if len(craft)==1]
            table = pd.concat([pd.DataFrame(crafts), pd.Series(notes)], axis=1)
            table.columns = ['In service', 'Aircraft', 'Notes']
            table = table[['Aircraft', 'In service', 'Notes']]
            table = transform_table(table, company, response)
            return table, 'table'
        # -------------------------------------

        else:

            info.append(soup.text.replace('\n', ' '))

        df['Parsed info'] = info
        df['Company'] = company
        df['Index'] = range(1, len(df) + 1)
        df.set_index(['Company', 'Index'], inplace=True)

        for col in df.columns:
            df[col] = df[col].replace(re.compile('\[.*?\]'), '').replace(re.compile('[0-9]*/'), '')

        got = True
        
        return df, 'text'
        
    except:
        
        return None, False

In [ ]:
def get_table(response, headers: dict, extra_page=False) -> tuple:
    """
    Get the table from company's wiki page response
    Function firstly extracts text of subtitle 'Fleet'
    through extract_txt(..) then extracts table from
    the received text with extract_table(...)

    Parameters
    ----------
    response: requests.Response
        Response from specific wikipage

    headers: dict
        headers to post on the wiki server

    extra_page: bool, default False
        flag, used to rerun function with searching
        additional fleet url and specifying different
        id_value parameter for extract_txt(..)

    """

    try:

        if not extra_page:
            text = extract_txt(response, headers, id_value=re.compile('Флот|Fleet|Operators_and_fleet|Current_fleet'))
            table = extract_table(text, response)

        else:
            text = extract_txt(response, headers)
            extra_soup = BeautifulSoup(text, 'html.parser')
            href = extra_soup.find('div', {"class": "hatnote navigation-not-searchable"}).find('a').get('href')
            new_url = f'https://en.wikipedia.org{href}'
            text = extract_txt(new_url, headers, id_value=re.compile('Current'), one_more_url=True)
            table = extract_table(text, new_url, one_more_url=True)
        
        got = True

    except:
        table = None
        got = False

    return table, got

## Logging the parse process

In [ ]:
def logging(company: str, response: str, urls: list, num, status) -> int:
    """
    Log the tray of parser's work state

    Parameters
    ----------
    company: str
        Name of the company to use in log
        
    response: requests.Response
        Response from specific wikipage
    
    responses: list
        List of all response from wikipage of companies that need to be parsed
        
    num: int
        Number of response in list responses
    
    status: str
        Some additional comments
        
    """
    
    print(company) 
    print(f'URL: {response.url}')
    print(f'{num+1}/{len(urls)} URLs parsed')
    print(f'Status: {status}')
    print(*['-' for i in range(20)])

## Parse exceptions handler

In [ ]:
def exception_handler(df: list) -> pd.DataFrame:
    """
    Work with the occuring exceptions of 
    obtained tables from some company wiki page

    Parameters
    ----------
    df: list
        List, containing possible desired DataFrame
        
    """
    
    na = pd.DataFrame()
    df1 = df[0] if len(df) > 0 else na
    try:    
        if 0 in df1.columns or 1 in df1.columns:
            for i in range(len(df)):
                df1 = df[i] if len(df) > 0 else na
                if 0 not in df1.columns or 1 not in df1.columns:
                    break
            else:
                df1 = df[0]
                df1 = df1.rename(columns=df1.iloc[0]).drop(df1.index[0])
    except:
        
        df1 = na
        
    if df1.empty:
        
        df1 = na
    
    return df1

## Separate function for parsing through one URL

In [ ]:
def parse(url: str, urls: list, headers=headers) -> Union[pd.DataFrame, None]:
    """
    Function, copying the parser logic, but with the single response
    
    Parameters
    ----------
    response (url): requests.Response
        Response from specific wikipage
    
    headers: dict
        Headers to post on wiki server
    
    """
        
    fleet = []
    corrupted = []
    exceptions = []

    df = []
    company = get_company_name(url)
    
    df, got = get_table(url, headers)

    if not got:
        
        df, got = get_table(url, headers, extra_page=True)
            
    if not got:
        
        df1, got = get_no_table(url, company, headers)
        
        if got == 'table':
            fleet.append(df1)   
            logging(company, url, urls, num, status='Non-table transformed to table and parsed')
            return df1
        
        if got == 'text':
            logging(company, url, urls, num, status='Non-table text exception')
            exceptions.append(df1)
            return df1
   
    if not got:
        logging(company, url, urls, num, status='Corrupted')
        print("Not able to parse data")
        corrupted.append(url.url)
        return None
    
    df1 = exception_handler(df)
    
    df1 = transform_table(df1, company, url)
        
    fleet.append(df1)   
    logging(company, url, urls, num, status='Table parsed')

    return df1

## Separate function for parsing through one company name

In [ ]:
def get_fleet(company: str, urls: list, headers: dict) -> Union[pd.DataFrame, None]:
    """
    Function, copying the parser and searcher logic, but with the single response
    
    Parameters
    ----------
    company: str
        Company name to search its page on wiki and parse through
    
    headers: dict
        Headers to post on wiki server
    
    """
    url = get_url(company)
    dataframe = parse(url, urls, headers)
    return dataframe

## Setting parse objects parameters

In [ ]:
def get_parse_urls(parse_obj=[], mode='urls'):
    """
    Function, getting urls to use in parser
    
    Parameters
    ----------
    parse_obj: list
        List of objects to proceed
        
    mode: {'urls', 'names'}, default 'urls'
        Mode pointing how to deal with parse_obj. If 'urls' specified, function serves as adapter 
        for get_list_companies(), if 'names' specified function searches names from parse_obj to
        get their wiki urls
    
    """
    
    if mode == 'urls':
        
        return parse_obj
    
    if mode == 'names':
        urls = [get_url(company) for company in tqdm.tqdm(parse_obj)]
        return urls
    

## Speeding up parsing with asynchronous requests

In [ ]:
def get_responses(urls, chunk_len=300, timeout=3, chunk=False):
    """
    Get responses from given urls. Uses asynchronous requests
    
    Parameters
    ----------
    urls: list
        List of urls to get responses from
        
    chunk_len: int, default 800
        Size of chunk to proceed if number of URLs is too high
        
    timeout: int, default 4
        How much to wait between each chunk load
        
    chunk: bool, default False
        Manually define if URLs should be splitted by chunks
    
    """
    
    def exception_handlerr(request, exception):
        print("Request failed", request.url)
        
    def chunks(xs, chunk_len=chunk_len):
        chunk_len = max(1, chunk_len)
        return (xs[i:i+chunk_len] for i in range(0, len(xs), chunk_len))
    
    if len(urls) > 1000:
        print('Number of URLs is to high to proceed further.\nSplitting URLs into chunks and loading them one by one...\n\n')
        chunk = True
        
    responses = []
    if chunk:
        for num, chunk in enumerate(chunks(urls, chunk_len=chunk_len)):
            rs = (grequests.get(u) for u in chunk)
            chunk_responses = grequests.map(rs, size=16, exception_handler=exception_handlerr)
            responses += chunk_responses
            print(f'Chunk #{num} of length {chunk_len} loaded, awaiting forced timeout of {timeout} seconds')
            time.sleep(timeout)
    else:
        rs = (grequests.get(u) for u in urls)
        responses = grequests.map(rs, size=16, exception_handler=exception_handlerr)
        
    corrupted = []
    responses = list(filter(lambda x: x, [response if response else corrupted.append(response) for response in responses])) 
    print(f'\nNumber of responded URLs that will be parsed: {len(responses)}\n')
    return responses, corrupted

## Concatenation of gathered tables

In [ ]:
def concat_tables(fleet: list, exceptions: list) -> Tuple[list,list]:
    """
    Concat received parsed and transformed 
    tables to get the full overview of aviacompanies' 
    fleet information

    Parameters
    ----------
    fleet: list
        list of special-form DataFrames with 
        data about aviacompanies' fleet
        
    exceptions: list
        list of special-form DataFrames with 
        data about aviacompanies' fleet,
        that contains information from no-table 
        aviacompanies' wiki pages
        
    """
    overall = pd.concat(fleet) if fleet else None
    overall_exp = pd.concat(exceptions) if exceptions else None
    
    return overall, overall_exp

## Loaders and handlers of external databases

In [ ]:
def load_icao():
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Accept-Language': 'en,ru-RU;q=0.9,ru;q=0.8,en-US;q=0.7',
        'Connection': 'keep-alive',
        # 'Content-Length': '0',
        'Origin': 'https://www.icao.int',
        'Referer': 'https://www.icao.int/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
        'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    response = rq.post('https://www4.icao.int/doc8643/External/AircraftTypes', headers=headers)
    data = response.json()
    icao_codes = pd.DataFrame(data)
    icao_codes.ManufacturerCode = icao_codes.ManufacturerCode.str.replace(' \([1-9]*\)', '', regex=True)
    icao_codes['FullName'] = icao_codes.ManufacturerCode.str.lower().str.capitalize() +" " + icao_codes.ModelFullName
    # a.columns[[9,5,0,4,1,6,7,8,2,3]]
    icao_codes = icao_codes[['FullName', 'ManufacturerCode', 'ModelFullName', 'Designator',
           'Description', 'AircraftDescription', 'EngineCount', 'EngineType',
           'WTC', 'WTG']]
    icao_codes = icao_codes.rename(columns={'Designator':'ICAO'})
    icao_codes.FullName = icao_codes.FullName.str.lower().str.capitalize()
    print('Dataset "icao" from https://www4.icao.int/doc8643/External/AircraftTypes has been gathered and processed')
    return icao_codes

In [ ]:
def load_mapper(preloaded=False):
    if preloaded:
        return pd.read_excel('mapper_codes.xlsx')
    mapper_codes = pd.read_html('https://www.greatcirclemapper.net/en/aircrafts.html')[0]
    mapper_codes['FullName'] = mapper_codes.Manufacturer + ' ' + mapper_codes.Name
    mapper_codes.Manufacturer = mapper_codes.Manufacturer.str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    mapper_codes.Name = mapper_codes.Name.str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    mapper_codes = mapper_codes[['Manufacturer', 'Name', 'IATA', 'Type']]
    manufacturers = mapper_codes.Manufacturer.tolist()
    names = mapper_codes.Name.tolist()
    mapper_codes = mapper_codes.fillna('-')
    resp = rq.get('https://www.greatcirclemapper.net/en/aircrafts.html')
    sp = BeautifulSoup(resp.text, 'html.parser')
    crafts_pages = []
    for craft in sp.find('table').find_all('strong'):
        if craft.find('a'):
            craft_page = craft.find('a').get('href')
            craft_page = f'https://www.greatcirclemapper.net/{craft_page}'
            crafts_pages.append(craft_page)
        else:
            crafts_pages.append(None)
    icaos_aircraft_mapper = []
    responses, _ = get_responses(crafts_pages)
    for response in tqdm.tqdm(responses):
        if response:
            soup_obj = BeautifulSoup(response.text, 'html.parser')
            icao_mapper = soup_obj.find('dt', string='ICAO').find_next_sibling().text.strip()
            icaos_aircraft_mapper.append(icao_mapper)
        else:
            icaos_aircraft_mapper.append(None)
    mapper_codes['ICAO'] = icaos_aircraft_mapper

    pat_split = ' / |, |/'
    frames = []
    for ind, obj in enumerate(manufacturers):
        manufacturers[ind] = re.split(pat_split, obj)

    mapper_codes['Manufacturer'] = manufacturers

    for ind, obj in enumerate(names):
        names[ind] = re.split(pat_split, obj)

    mapper_codes['Name'] = names
    mapper_codes.IATA = mapper_codes.IATA.str.split('      ')
    mapper_codes.Type = mapper_codes.Type.str.split('      ')
    mapper_codes.ICAO = mapper_codes.ICAO.str.split('      ')
    for row in range(len(mapper_codes)):
        for element in itertools.product(*mapper_codes.values[row]):
            frames.append(pd.DataFrame([element], columns = mapper_codes.columns))

    mapper_codes = pd.concat(frames)
    mapper_codes.Manufacturer = mapper_codes.Manufacturer.str.replace(' Aircraft', ' ')
    mapper_codes['FullName'] = mapper_codes.Manufacturer + ' ' + mapper_codes.Name
    mapper_codes = mapper_codes.drop_duplicates()
    mapper_codes = mapper_codes[['FullName', 'Manufacturer', 'Name','ICAO', 'IATA', 'Type']]
    mapper_codes.FullName = mapper_codes.FullName.str.lower().str.capitalize()
    mapper_codes = mapper_codes[~mapper_codes['FullName'].duplicated()]
    mapper_codes.to_excel('mapper_codes.xlsx', index=False)
    print('Dataset "mapper" from https://www.greatcirclemapper.net/en/aircrafts.html has been gathered and processed')
    return mapper_codes.reset_index(drop=True)

In [ ]:
def load_wiki1():
    wiki_codes1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_aircraft_type_designators')[0]
    wiki_codes1.columns = ['ICAO', 'IATA', 'FullName']
    wiki_codes1 = wiki_codes1[['FullName', 'ICAO', 'IATA']]
    wiki_codes1.FullName = wiki_codes1.FullName.str.lower().str.capitalize()
    wiki_codes1 = wiki_codes1[~wiki_codes1['FullName'].duplicated()]
    print('Dataset "wiki1" from https://en.wikipedia.org/wiki/List_of_aircraft_type_designators has been gathered and processed')
    return wiki_codes1

In [ ]:
def load_wiki2():
    resp2 = rq.get('https://en.wikipedia.org/wiki/List_of_civil_aircraft')
    sp2 = BeautifulSoup(resp2.text, 'html.parser')
    wiki_codes2 = []
    for i in sp2.find_all('a', {'href': re.compile('/wiki/')})[2:1856]:
        wiki_codes2.append(i.text)
    wiki_codes2 = pd.DataFrame(wiki_codes2, columns=['FullName'])
    wiki_codes2.FullName = wiki_codes2.FullName.str.lower().str.capitalize()
    wiki_codes2 = wiki_codes2[~wiki_codes2['FullName'].duplicated()]
    print('Dataset "wiki2" from https://en.wikipedia.org/wiki/List_of_civil_aircraft has been gathered and processed')
    return wiki_codes2

In [ ]:
def load_wiki3():
    resp3 = rq.get('https://en.wikipedia.org/wiki/List_of_aircraft_by_date_and_usage_category')
    sp3 = BeautifulSoup(resp3.text, 'html.parser')
    wiki_codes3 = []
    for i in sp3.find('table').find_next('table').find_next('table').find_next('table').find_next('table').find_all('a', {'href': re.compile('/wiki/')}):
        wiki_codes3.append(i.text)
    wiki_codes3 = pd.DataFrame(wiki_codes3, columns=['FullName'])
    wiki_codes3.FullName = wiki_codes3.FullName.str.lower().str.capitalize()
    wiki_codes3 = wiki_codes3[~wiki_codes3['FullName'].duplicated()]
    print('Dataset "wiki3" from https://en.wikipedia.org/wiki/List_of_aircraft_by_date_and_usage_category has been gathered and processed')
    return wiki_codes3


In [ ]:
def load_wiki4():
    df = pd.read_html('https://en.wikipedia.org/wiki/List_of_airline_codes')[0]
#     df = df.rename(columns={'IATA':'IATA_airline', "ICAO":"ICAO_airline", "Airline":"FullName"})
    df = df.rename(columns={"Airline":"FullName"})
    df.FullName = df.FullName.str.lower().str.capitalize()
    df = df[~df.duplicated()]
    df = df[~df.FullName.duplicated()]
    print('Dataset "code" from https://en.wikipedia.org/wiki/List_of_airline_codes has been gathered and processed')
    return df

In [ ]:
def load_fan(preloaded=False):
    if preloaded:
            return pd.read_excel('fan_codes.xlsx')
    tables = []
    urls = []
    for page in range(1, 275):
        urls.append(f'https://www.aviationfanatic.com/ent_list.php?ent=4&pg={page}')
    responses, _  = get_responses(urls)
    for response in responses:
        tables.append(pd.read_html(response.text)[0][:-2])
    fan_codes = pd.concat(tables)
    fan_codes.columns = ['#', 'ID', 'Manufacturer', 'FullName',
           '# of related pictures', 'ICAO',
           'Manufacturer country', 'Category', 'Role', 'Engine type', 'Engines',
           'WTC', 'Seats', 'First flight', 'Last manufactured', 'Total # built',
           'Info (external)', '# of aircraft in DB', '# of related collections',
           '# of related user comments', 'Unnamed: 20']
    fan_codes = fan_codes[['ID', 'Manufacturer', 'FullName', 'ICAO',
           'Manufacturer country', 'Category', 'Role', 'Engine type', 'Engines','WTC']]
    fan_codes.FullName = fan_codes.FullName.str.lower().str.capitalize()
    fan_codes = fan_codes[~fan_codes['FullName'].duplicated()]
    fan_codes.to_excel('fan_codes.xlsx', index=False)
    print('Dataset "fan" from https://www.aviationfanatic.com/ent_list.php?ent=4 has been gathered and processed')
    return fan_codes.reset_index(drop=True)

In [ ]:
def load_mil():
    rspn = rq.get("https://www.militaryfactory.com/aircraft/indexMAIN.php").text
    sp = BeautifulSoup(rspn, 'html.parser')
    mil_codes = []
    for i in sp.find_all('span', {"class":"textLarge textBold textDkGray"}):
        mil_codes.append(i.text)
    mil_codes = pd.Series(mil_codes).replace(' \((.*?)\)', '', regex=True).str.strip().tolist()
    mil_codes = pd.DataFrame(mil_codes, columns=['FullName'])
    mil_codes.FullName = mil_codes.FullName.str.lower().str.capitalize()
    mil_codes = mil_codes[~mil_codes['FullName'].duplicated()]
    print('Dataset "military" from https://www.militaryfactory.com/aircraft/indexMAIN.php has been gathered and processed')
    return mil_codes

In [ ]:
def load_ssim():
    ssim_codes = pd.read_html('http://wiki.aviabit.ru/doku.php?id=pub:ssim._apendix_a')[0]
    ssim_codes.columns = ['FullName', 'IATA', 'Group IATA', 'Cate', 'ICAO']
    ssim_codes.FullName = ssim_codes.FullName.str.lower().str.capitalize()
    ssim_codes.FullName = ssim_codes.FullName.str.replace('passenger', '', regex=False)
    ssim_codes = ssim_codes[~ssim_codes['FullName'].duplicated()]
    print('Dataset "iata" from http://wiki.aviabit.ru/doku.php?id=pub:ssim._apendix_a has been gathered and processed')
    return ssim_codes

In [ ]:
def load_flug_icao():
    flug_codes = pd.concat(pd.read_html('http://www.flugzeuginfo.net/table_accodes_en.php'))
    flug_codes = flug_codes.rename(columns={'Type/Model': 'Name', 'Wake':'WTC'})
    flug_codes.Manufacturer = flug_codes.Manufacturer.str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    flug_codes.Name = flug_codes.Name.str.replace('(', '', regex=False).str.replace(')', '', regex=False)

    manufacturers = flug_codes.Manufacturer.tolist()
    names = flug_codes.Name.tolist()
    flug_codes = flug_codes.fillna('-')
    pat_split = ' / |, |/'
    frames = []
    for ind, obj in enumerate(manufacturers):
        manufacturers[ind] = re.split(pat_split, obj)
    flug_codes['Manufacturer'] = manufacturers

    for ind, obj in enumerate(names):
        names[ind] = re.split(pat_split, obj)
    flug_codes['Name'] = names

    flug_codes.ICAO = flug_codes.ICAO.str.split('      ')
    flug_codes.WTC = flug_codes.WTC.str.split('      ')
    for row in range(len(flug_codes)):
        for element in itertools.product(*flug_codes.values[row]):
            frames.append(pd.DataFrame([element], columns = flug_codes.columns))

    flug_codes = pd.concat(frames)
    flug_codes['FullName'] = flug_codes.Manufacturer + ' ' + flug_codes.Name
    flug_codes = flug_codes[['FullName', 'Manufacturer', 'Name', 'ICAO', 'WTC']]
    flug_codes = flug_codes[~flug_codes['FullName'].duplicated()]
    print('Dataset "flug_icao" from http://www.flugzeuginfo.net/table_accodes_en.php has been gathered and processed')
    return flug_codes.reset_index(drop=True)

In [ ]:
def load_flug_iata():
    flug_codes = pd.read_html('http://www.flugzeuginfo.net/table_accodes_iata_en.php')[0]
    flug_codes = flug_codes.rename(columns={'Type/Model': 'Name', 'Wake':'WTC'})
    flug_codes.Manufacturer = flug_codes.Manufacturer.str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    flug_codes.Name = flug_codes.Name.str.replace('(', '', regex=False).str.replace(')', '', regex=False)

    manufacturers = flug_codes.Manufacturer.tolist()
    names = flug_codes.Name.tolist()
    flug_codes = flug_codes.fillna('-')
    pat_split = ' / |, |/'
    frames = []
    for ind, obj in enumerate(manufacturers):
        manufacturers[ind] = re.split(pat_split, obj)
    flug_codes['Manufacturer'] = manufacturers

    for ind, obj in enumerate(names):
        names[ind] = re.split(pat_split, obj)
    flug_codes['Name'] = names

    flug_codes.IATA = flug_codes.IATA.str.split('      ')
    flug_codes.WTC = flug_codes.WTC.str.split('      ')
    for row in range(len(flug_codes)):
        for element in itertools.product(*flug_codes.values[row]):
            frames.append(pd.DataFrame([element], columns = flug_codes.columns))

    flug_codes = pd.concat(frames)
    flug_codes['FullName'] = flug_codes.Manufacturer + ' ' + flug_codes.Name
    flug_codes = flug_codes[['FullName', 'Manufacturer', 'Name', 'IATA', 'WTC']]
    flug_codes = flug_codes[~flug_codes['FullName'].duplicated()]
    print('Dataset "flug_iata" from http://www.flugzeuginfo.net/table_accodes_iata_en.php has been gathered and processed')
    return flug_codes.reset_index(drop=True)

In [ ]:
def load_av():
    av_codes = pd.read_html('https://www.avcodes.co.uk/acrtypes.asp')[0].rename(columns={'IATACode':"IATA", 'ICAOCode':'ICAO', 'Manufacturer and Aircraft Type / Model': 'FullName'})
    av_codes = av_codes[~av_codes['FullName'].duplicated()]
    print('Dataset "av" from https://www.avcodes.co.uk/acrtypes.asp has been gathered and processed')
    return av_codes[~av_codes['FullName'].duplicated()]

In [ ]:
def load_av_airlines(preloaded=False):
    if preloaded:
        return pd.read_excel('av_airlines.xlsx')
    id_start = 23676
    id_finish = 29983
    urls = [f'https://www.avcodes.co.uk/details.asp?ID={ID}' for ID in range(id_start, id_finish+1)]
    resps = get_responses(urls)
    rows = []
    for num, resp in enumerate(resps):
        if not resp:
            continue
        soup = BeautifulSoup(resp.text, 'html.parser')
        elements = []
        for element in soup.find_all('td'): 
            hhh = element.text.strip().split(':') if element.text else ['']
            hhh = hhh[1] if len(hhh)>1 else hhh[0]
            elements.append(hhh)
        if len(elements)==14:
            elements.insert(1, '')
        rows.append(elements)
    
    columns = ['name', 'logo', 'FullName', 'IATA', 'ICAO', 'ICAO_callsign',
            'IATA_accounting', 'IATA_prefix', 'Country', 'Website', 'flag',
            'Founded', 'commences_ops', 'ceased_ops', 'remarks']
    av_airlines = pd.DataFrame(rows, columns=columns)
    av_airlines.Website = av_airlines.Website.str.replace('Website URL', '')
    av_airlines.FullName = av_airlines.FullName.str.lower().str.capitalize()
    av_airlines.to_excel('av_airlines.xlsx', index=False)
    print('Dataset "av_airlines" from https://www.avcodes.co.uk/details.asp has been gathered and processed')
    return av_airlines

In [ ]:
def load_flug_airlines():
    df = pd.concat(pd.read_html('http://www.flugzeuginfo.net/table_airlinecodes_airline_en.php'))
    df = df.rename(columns={'Airline':'FullName'})
    df.FullName = df.FullName.str.lower().str.capitalize()
    print('Dataset "flug_airlines" from http://www.flugzeuginfo.net/table_airlinecodes_airline_en.php has been gathered and processed')
    return df

In [ ]:
def load_aircodes():
    df = pd.read_html('https://airlinecodes.info/icao')[0]
    df = df.rename(columns={"NAME":"FullName"})
    df.FullName = df.FullName.str.lower().str.capitalize()
    df = df[~df.duplicated()]
    df = df[~df.FullName.duplicated()]
    print('Dataset "aircodes" from https://airlinecodes.info/icao has been gathered and processed')
    return df

## Separate function for loading all external databases

In [ ]:
def load_databases():
    ssim_codes = load_ssim()
    icao_codes = load_icao()
    wiki_codes1 = load_wiki1()
    wiki_codes2 = load_wiki2()
    wiki_codes3 = load_wiki3()
    fan_codes = load_fan(preloaded=True)
    mapper_codes = load_mapper(preloaded=True)
    mil_codes = load_mil()
    flug_icao = load_flug_icao()
    flug_iata = load_flug_iata()
    av_codes = load_av()
    all_codes = {
             'iata':ssim_codes, 'icao':icao_codes, 'wiki1': wiki_codes1, 
             'wiki2':wiki_codes2, 'wiki3':wiki_codes3, 'fan': fan_codes, 
             'mapper': mapper_codes, 'military': mil_codes, 'flug_iata': flug_iata,
             'flug_icao':flug_icao, 'av': av_codes
            }
    return all_codes

## Modification difflib.get_close_matches() 

In [ ]:
def get_close_matches(word, possibilities, n=1, cutoff=0.6):
    """
    Modification of difflib.get_close_matches()
    Now also return scores, not only the matches
    
    Parameters
    ----------
    word: str
        Word to find close matches
    
    possibilities: Iterable
        Iterable object, containing possible matches
    
    cutoff: float, default 0.6
        Minimal appropriate match score for matches
    """

    if not n >  0:
        raise ValueError("n must be > 0: %r" % (n,))
    if not 0.0 <= cutoff <= 1.0:
        raise ValueError("cutoff must be in [0.0, 1.0]: %r" % (cutoff,))
    result = []
    s = SequenceMatcher()
    s.set_seq2(word)
    for x in possibilities:
        s.set_seq1(x)
        if s.real_quick_ratio() >= cutoff and \
           s.quick_ratio() >= cutoff and \
           s.ratio() >= cutoff:
            result.append((s.ratio(), x))

    # Move the best scorers to head of list
    result = _nlargest(n, result)
    # Strip scores for the best n matches
    return [(score, x) for score, x in result]

## Related function to get_close_matches() that returns only one match with its score (cutoff)

In [ ]:
def get_closest_match(x, other, cutoff):
    matches = get_close_matches(x, other, n=1, cutoff=cutoff)
    return matches[0][0], matches[0][1] if matches else None

## Main algorithm of matching fuzzy values

In [ ]:
def fuzzy_merge(df1, df2, left_on, right_on, how='left', cutoff=0.65):
    """
    Make a merge on fuzzy matches
    
    Parameters
    ----------
    df1: pd.DataFrame
        Left DataFrame to fuzzy merge on
        
    df2: pd.DataFrame
        Right DataFrame to fuzzy match with
        
    left_on: str
        Left DataFrame column name whose values to match
        
    right_on: str
        Right DataFrame column name whose values to fuzzy merge
        
    how: str, default 'left'
        how to proceed merge

    cutoff: int, default 0.65
        Minimal appropriate match score for matches
    """
    df_other= df1.copy()

    left = [None for i in range(len(df1[left_on]))]
    scores = [None for i in range(len(df1[left_on]))]
    df2 = df2.groupby(by=right_on).first().reset_index().copy()

    for ind, x in enumerate(df_other[left_on]):
        try:
            score, match = get_closest_match(x, df2[right_on][df2[right_on].str.contains(x.split()[0], na=False)], cutoff)
        except Exception as Ex:
            score = None
            match = None
        if match and not left[ind]:
            left[ind] = match
            scores[ind] = score
     
    df1[right_on] = left
    df1['cutoffs'] = scores

    return df1.merge(df2, on=right_on, how='left')

## Deriving and processing column that needs to be fuzzy matched

In [ ]:
def derive_match_col(df, col_to_derive, change_values_of, how_to_name='Crafts'):
    """
    Derive and process match column to make fuzzy merge on
    
    Parameters
    ----------
    df: pd.DataFrame
        DataFrame from which to get column
    
    col_to_derive: str
        Name of column to use for matching
        
    change_values_of: list,
        List of values that should be changed for better matching
        
    how_to_name: str
        Name of derived column
    
    """
    
    
    proxy = df.copy()
    
    proxy[how_to_name] = overall[col_to_derive]
    proxy_deriv = proxy[how_to_name].to_frame().reset_index(drop=True)
    proxy_deriv[how_to_name] = proxy_deriv[how_to_name].str.lower().str.capitalize()
    
    if how_to_name == "Crafts":
        for change_value in change_values_of:
            proxy_deriv[how_to_name][proxy_deriv[how_to_name].str.contains(change_value, na=False)] = \
            proxy_deriv[how_to_name][proxy_deriv[how_to_name].str.contains(change_value, na=False)] \
            .str.replace('(-100)', '', regex=True).str.replace('-200', '').str.replace('-231','') \
            .str.replace('Ан', 'Антонов ан').str.replace('Ми', 'Мил ми')
    
    trans = lambda x: transliterate.translit(x, 'ru', reversed=True) if x and type(x)==str else np.nan
    proxy_deriv[how_to_name] = proxy_deriv[how_to_name].apply(trans)
    
    return proxy_deriv

## Match engine algorithm that performs analysis through all loaded databases

In [ ]:
def analyze(main_frame, analyzers, left_on='Crafts', right_on='FullName', cutoff=0.65):
    """
    Find matches throughout several datasets to find best matching value
    
    Parameters
    ----------
    main_frame: pd.DataFrame
        1-d DataFrame with column to match
    
    analyzers: list
        List, containing tuples of dataset and their specifications
        
    left_on: str
        Left DataFrame column name whose values to match
        
    right_on: str
        Right DataFrame column name whose values to fuzzy merge

    cutoff: int, default 0.65
        Minimal appropriate match score for matches
    
    """
    
    for num, analyzer in enumerate(analyzers):
        start = datetime.now()
        data = analyzer[1].copy()
        print(f"Iteration #{num}")
        print(f'Set: "{analyzer[0]}",  lenght: {len(data)}')
        data.FullName = data.FullName.str.lower().str.capitalize().reset_index(drop=True)
        main_frame = fuzzy_merge(main_frame, data, left_on=left_on, right_on=right_on, how='left', cutoff=cutoff)
        main_frame.rename(columns={'FullName': f'FullName_{num}', 'cutoffs': f'cutoffs_{num}'}, inplace=True)
        print(f"Time taken: {datetime.now()-start}\nFrame lenght: {len(main_frame)}")
        print(*['-' for i in range(20)])
    return main_frame
        

## Derivation of best cutoff(score) through all cutoffs for all observations

In [ ]:
def derive_best_score(df, score_col_name='best_cutoff'):
    if df.dropna(thresh=2).empty:
        df[score_col_name] = np.nan
        return None
    cutoff_cols = [column for column in df.columns if 'cutoffs' in column]
    for col in cutoff_cols:
        df[col] = pd.to_numeric(df[col], errors='ignore')
    df[score_col_name] = df[cutoff_cols].idxmax(axis=1)
    df

## Derivation of best matches and their original databases indices for all observations

In [ ]:
def get_info_best(df, best_cutoff_col):
    if df.dropna(thresh=2).empty:
        df['best_match'] = np.nan
        df['best_match_index'] = np.nan
        df['codes_base_num'] = np.nan
        return None
    df['best_match'] = df[best_cutoff_col][df[best_cutoff_col].str.contains('_', na=False)].replace('cutoffs', 'FullName', regex=True)
    df['best_match_index'] = (df['best_match'].str[-1].astype(float)*2 + 1).astype(int, errors='ignore')
    df['codes_base_num'] = df['best_match'].str[-1]
    return df

## Derivation original databases of best matches

In [ ]:
def find_db(df, db_ind_col, db_list):
    orig_database = []
    for row in df.index:
        try:
            key = db_list[int(df.loc[row, db_ind_col])][0]
        except:
            key = np.nan
            
        orig_database.append(key)
    
    return orig_database

## Check for designator (ICAO, IATA) availability in original databases for all observations

In [ ]:
def desig_availability(df, db_col, db_available):
    desig_truth = []
    for row in df.index:
        dataset = df.loc[row, db_col]
        try:
            have_desig = db_available[dataset]
        except:
            have_desig = False
            
        desig_truth.append(have_desig)
    
    return desig_truth

## Derivation of designator (ICAO, IATA) values for all observations

In [ ]:
def get_desig_values(df, db_col, desig_truth_col, dbs_dct, desig):
    
    desig_codes = []
    
    for row in df.index:
    
        dataset = df.loc[row, db_col]

        if df.loc[row, desig_truth_col] == True:
            match_col = df.loc[row, 'best_match']
            match_value = df.loc[row, match_col]
            desig_value = dbs_dct[dataset].loc[dbs_dct[dataset]['FullName']==match_value, desig]
            try:
                desig_value = desig_value.item() if len(desig_value) else np.nan
            except:
                desig_value = desig_value.iloc[0] if len(desig_value) else np.nan
        
        else:
            desig_value = np.nan
            
        desig_codes.append(desig_value)
        
    return desig_codes

## Obtaining values (best_match/best_cutoff) for all observations

In [ ]:
def get_values(df, column):
    if df.dropna(thresh=4).empty:
        values = np.nan
        return values
    values = []
    for row in df.index:
        val_col = df.loc[row, column]
        if val_col is not np.nan:
            value = df.loc[row, val_col]
        else:
            value = np.nan
        values.append(value)
    return values

## Rematching algorithm that tries to guess unmatched observations one more time 

In [ ]:
def rematching(df, dbs_dct, left_on, right_on, skip_list, icao_list, icaos_aircraft, iatas_aircraft):
    print('\nRematching unfilled values...\n\n')
    rematch_vals = []
    for row in df.index:
        no_icao = not df.loc[row, 'have_icao']
        rematch_vals.append(no_icao)
    rematch_rows = df[rematch_vals].copy()
    if rematch_rows.dropna(thresh=4).empty:
        return pd.DataFrame(columns=[left_on, 'best_cutoff', 'orig_database', 'best_match',
                                     'best_match_index', 'codes_base_num', 'have_icao', 'have_iata',
                                     'ICAO', 'IATA', 'researched_value', 'cutoff']
                           )

    new_matches = []
    for row in rematch_rows.index:

        new_match_col = rematch_rows.loc[row, 'best_match']
        if new_match_col is not np.nan:
            new_match = rematch_rows.loc[row, new_match_col]
        else:
            new_match = np.nan

        new_matches.append(new_match)
        
    rematch_rows['rematch_values'] = new_matches
    
    rematch_old_index = rematch_rows.index
    
    rematch_crafts = rematch_rows.rematch_values.to_frame().rename(columns={'rematch_values':'Crafts'})
    
    dbs_rematch = [(key, dbs_dct[key]['FullName'].to_frame()) for key in dbs_dct \
                   if key not in skip_list and key in icao_list]

    rematch_df = analyze(rematch_crafts, dbs_rematch, left_on, right_on, cutoff=0.65)
    
    rematch_df.index = rematch_old_index
    
    derive_best_score(rematch_df)
    
    rematch_df = get_info_best(rematch_df, 'best_cutoff')
    
    rematch_df['orig_database'] = find_db(rematch_df, 'codes_base_num', dbs_rematch)
    
    rematch_df['have_icao'] = desig_availability(rematch_df, 'orig_database', icaos_aircraft)
    rematch_df['have_iata'] = desig_availability(rematch_df, 'orig_database', iatas_aircraft)
    
    rematch_df['ICAO'] = get_desig_values(rematch_df, 'orig_database', 'have_icao', dbs_dct, 'ICAO')
    rematch_df['IATA'] = get_desig_values(rematch_df, 'orig_database', 'have_iata', dbs_dct, 'IATA')
    
    rematch_df['researched_value'] = get_values(rematch_df, 'best_match')
    rematch_df['cutoff'] = get_values(rematch_df, 'best_cutoff')
    return rematch_df

## Matching algorithm that contains all processing and values derivation steps

In [ ]:
def matching(main_frame, dbs_dct, left_on, right_on, skip_list, icao_list, icaos_aircraft, iatas_aircraft, cutoff=0.65):
    print(f"Cutoff: {cutoff}\n\n")
    dfs = [(key, dbs_dct[key]['FullName'].to_frame()) for key in dbs_dct if key not in skip_list]
    
    analysis = analyze(main_frame, dfs[:], left_on, right_on, cutoff=cutoff)
    df = analysis.copy()

    derive_best_score(df)

    df = get_info_best(df, 'best_cutoff')

    df['orig_database'] = find_db(df, 'codes_base_num', dfs)

    df['have_icao'] = desig_availability(df, 'orig_database', icaos_aircraft)
    df['have_iata'] = desig_availability(df, 'orig_database', iatas_aircraft)
    df['ICAO'] = get_desig_values(df, 'orig_database', 'have_icao', dbs_dct, 'ICAO')
    df['IATA'] = get_desig_values(df, 'orig_database', 'have_iata', dbs_dct, 'IATA')
    df['researched_value'] = get_values(df, 'best_match')
    df['cutoff'] = get_values(df, 'best_cutoff')
    
    analysis_rematch = rematching(df, dbs_dct, left_on, right_on, skip_list, icao_list, icaos_aircraft, iatas_aircraft)
    
    for row in analysis_rematch.index:
        df.loc[row, 'orig_database'] = analysis_rematch.loc[row, 'orig_database']
        df.loc[row, 'ICAO'] = analysis_rematch.loc[row, 'ICAO']
        df.loc[row, 'IATA'] = analysis_rematch.loc[row, 'IATA']
        df.loc[row, 'researched_value'] = analysis_rematch.loc[row, 'researched_value']
        df.loc[row, 'cutoff'] = analysis_rematch.loc[row, 'cutoff']
    
    for row in df.index:
    
        dataset = df.loc[row, 'orig_database']
        
        if not pd.isnull(dataset):

            df.loc[row, 'orig_database'] = database_url[dataset]

        else:

            pass
        
    df = df[[left_on, 'best_match', 'best_match_index', 'codes_base_num',
             'have_icao', 'have_iata', 'ICAO', 'IATA', 'researched_value',
             'cutoff', 'orig_database']]
    
    return df
    

## Add info on received ICAO and IATA designators

In [ ]:
def add_info(df, dbs_dct):
    fan_cols = ['Manufacturer', 'FullName', 'ICAO', 'Manufacturer country', 'Category', 'Role']
    keep_cols = ['Aircraft ICAO', 'Aircraft IATA', 'ManufacturerCode', 'Description', 
                 'AircraftDescription', 'EngineCount', 'EngineType', 'WTC', 'WTG', 
                 'Role', 'researched_aircraft_value', 'aircraft_cutoff', 'aircrafts_database']
    df_merged = pd.merge(df, dbs_dct['icao'].groupby(by='ICAO').first() \
                         .reset_index(), left_on='Aircraft ICAO', right_on='ICAO', how='left', suffixes=(None, '_check'))
    
    df_merged1 = pd.merge(df_merged, dbs_dct['iata'].groupby(by='IATA').first() \
                          .reset_index(), left_on='Aircraft IATA', right_on='IATA', how='left', suffixes=(None, '_check'))
    
    df_merged2 = pd.merge(df_merged1, dbs_dct['fan'][fan_cols].groupby(by='ICAO').first() \
                          .reset_index(), left_on='Aircraft ICAO', right_on='ICAO', how='left')
    
    df_merged2 = df_merged2[keep_cols]
    return df_merged2

## Pretty printing

In [ ]:
def appendix(db, db_urls, status):
    """
    Pretty print some specified load status information of datasets
    
    Parameters
    ----------
    
    db: str
        Name of dataset
    
    db_urls: dict
        Dictionary, containing urls of all datasets
        
    status: bool
        Status of dataset load
    """
    appendix1 = ' ' if status else ' not '
    appendix2 = 'loaded from the disk. (reload dataset from time to time to update data)' if status else 'parsed that would take some time.'
    return db_urls[db], appendix1, appendix2

## Creation of the final report files

In [ ]:
def create_report(overall, matches_full, overall_exp, corrupted, index_visualize=True) -> None:
    """
    Create Excel report about aviacompanies' fleet 
    with making directory and specifying date and time

    Parameters
    ----------
    overall: pd.DataFrame
        DataFrame with information 
        about aviacompanies' fleet
        
    matches_airlines: pd.DataFrame
        DataFrame of airlines matches
        
    matches_full: pd.DataFrame
        DataFrame of aircrafts matches with added information
        
    overal_exp: pd.DataFrame
        DataFrame with information 
        about parsed wiki pages exceptions 
        of aviacompanies' fleet
        
    corrupted: list
        List of all corrupted URLs
        
    """
    
    t_date = datetime.now().strftime('%d_%m_%Y, %H.%M')

    Path(f"{path}/report_{t_date}").mkdir(parents=True, exist_ok=True)
    
    columns_order = [
        'Company', 'Airline ICAO', 'Airline IATA', 'Callsign', 'Wiki URL', 'Aircraft',
        'In service', 'Orders', 'Total_Passengers', 'Notes','Aircraft ICAO',
        'Aircraft IATA', 'ManufacturerCode', 'Description', 'AircraftDescription', 
        'EngineCount', 'EngineType', 'WTC', 'WTG', 'Role', 'researched_aircraft_value', 
        'aircraft_cutoff','aircrafts_database'
    ]
    
    os.chdir(f"{path}/report_{t_date}")
    if index_visualize:
        multiindex_cols = ['Company', 'Airline ICAO', 'Airline IATA', 'Callsign', 'Wiki URL', 'Aircraft', 'In service', 'Orders']
    
    if corrupted: 
        pd.DataFrame(corrupted, columns=['Corrupted URLs']).to_excel(f'corrupted_URLs_{t_date}.xlsx', index=False)
    
    if overall is not None:
        final_table = pd.concat([overall.reset_index(drop=True), matches_full], axis=1)[columns_order]
        
        if index_visualize:
            final_table.set_index(multiindex_cols, inplace=True)
            
        final_table.to_excel(f'fleet_report_{t_date}.xlsx', index=False)
        
    if overall_exp is not None: 
        overall_exp.to_excel(f'fleet_report_exceptions_{t_date}.xlsx')
    
    print(f'Reports were created and stored at {os.getcwd()}')

---------

---------

---------

<!-- --------- -->

---------

---------

---------

<h1 style="font-size:100px">Parser</h1>

In [ ]:
script_started = datetime.now()

In [ ]:
os.getcwd()

### 1. Setting up of initial data and getting parse urls
1. In a variable **source** write down what source to parse objects from

    possible options: **{'pulkovo', 'us', 'europe', 'cac', 'africa', 'asia', 'northamerica', 'oceania', 'southamerica', 'all', 'test', 'national', 'pulkovo_preloaded'}**


2. In a variable **mode**  write what type of data to gather

    possible options: **{'urls', 'names'}**

NOTE! You can use only 'names' option in a variable **mode** if the source==**'pulkovo'**

NOTE! You can use source==**'pulkovo_preloaded'** only if you have appropriate dataset

In [ ]:
source = 'test'
mode = 'urls'
parse_obj = get_list_companies(source=source, mode=mode, headers=headers)
urls = get_parse_urls(parse_obj=parse_obj, mode=mode)

In [ ]:
%%time
responses, corrupted_ = get_responses(urls)

#### List of unresponded corrupted URLs

In [ ]:
corrupted_

In [ ]:
set([i.status_code if i is not None else '' for i in corrupted_])

### 2. Main fleet parser (uses urls variable to iterate and parse)

In [ ]:
%%time
fleet = []
corrupted = []
exceptions = []

for num, response in enumerate(responses[:]):

    df = []
    company = get_company_name(response)
    
    df, got = get_table(response, headers)

    if not got:
        
        df, got = get_table(response, headers, extra_page=True)
            
    if not got:
        
        df1, got = get_no_table(response, company, headers)
        
        if got == 'table':
            fleet.append(df1)   
            logging(company, response, responses, num, status='Non-table transformed to table and parsed')
            continue
        
        if got == 'text':
            logging(company, response, responses, num, status='Non-table text exception')
            exceptions.append(df1)
            continue
   
    if not got:
        logging(company, response, responses, num, status='Corrupted')
        corrupted.append(response.url)
        continue
    
    df1 = exception_handler(df)
    
    df1 = transform_table(df1, company, response)
        
    fleet.append(df1)   
    logging(company, response, responses, num, status='Table parsed')
    
success = len(fleet)+len(exceptions)
print('', '', '', sep='\n')
print(f'{success}/{len(responses)} companies successfully parsed')

#### Statistics for parsed urls
* len(fleet) -- stands for the number of processed tables with fleet information
* len(exception) -- stands for the number of received non-table fleet info that for the moment could not be processed further
* len(corrupted) -- stands for the number of URLs that could not be parsed due to the different reasons
* len(corrupted_) -- stands for the number of URLs that could not be parsed due to the absence of page on website

In [ ]:
len(fleet), len(exceptions), len(corrupted), len(corrupted_)

#### List of corrupted URLs during the processing

In [ ]:
corrupted

### 3. Concatenation of received parsed tables with fleet info

In [ ]:
overall, overall_exp = concat_tables(fleet, exceptions)

### 4. Setting up the storage and databases paths

In [ ]:
# Storage path

path = "C:\\Users\\Asus\\Downloads\\parser\\reports"
os.chdir(path)
os.getcwd()

In [ ]:
# Path to databases

db_path = "datasets/"

In [ ]:
try:
    shutil.copy(f"{db_path}fan_codes.xlsx", f"{path}")
    preloaded_fan=True
except:
    preloaded_fan=False
to_print = appendix('fan', database_url, preloaded_fan)
print(f'Heavy dataset {to_print[0]} is{to_print[1]}preloaded.\nIt is going to be {to_print[2]}\n')

try:    
    shutil.copy(f"{db_path}mapper_codes.xlsx", f"{path}")
    preloaded_map=True
except:
    preloaded_map=False   
to_print = appendix('mapper', database_url, preloaded_map)
print(f'Heavy dataset {to_print[0]} is{to_print[1]}preloaded.\nIt is going to be {to_print[2]}\n')
    
try:    
    shutil.copy(f"{db_path}av_airlines.xlsx", f"{path}")
    preloaded_av=True
except:
    preloaded_av=False    
to_print = appendix('av_airlines', database_url, preloaded_av)
print(f'Heavy dataset {to_print[0]} is{to_print[1]}preloaded.\nIt is going to be {to_print[2]}\n')

### 5. Loading and processing external databases
####  Any of realisations might be used

In [ ]:
%%time
# all_codes = load_databases()
all_codes_airlines = {
                      'code': load_wiki4(), 
                      'av_airlines': load_av_airlines(preloaded=preloaded_av),
                      'flug_airlines': load_flug_airlines(),
                      'aircodes': load_aircodes()
                    }

print('\nAirlines datasets were loaded.\n')

In [ ]:
%%time
all_codes = {
         'iata': load_ssim(), 'icao': load_icao(), 'wiki1': load_wiki1(), 
         'wiki2': load_wiki2(), 'wiki3': load_wiki3(), 'fan': load_fan(preloaded=preloaded_fan), 
         'mapper': load_mapper(preloaded=preloaded_map), 'military': load_mil(), 'flug_iata': load_flug_iata(),
         'flug_icao': load_flug_icao(), 'av': load_av()
        }

print('\nAircrafts datasets were loaded.\n')

### 6. Derivation and processing the column on which the match will be searched

In [ ]:
main_frame_aircrafts = derive_match_col(overall, col_to_derive='Aircraft', change_values_of=change_values_of_aircrafts)

### 7. Advanced fuzzy matching of ICAO and IATA codes to the aircrafts

In [ ]:
matches_aircrafts = matching(main_frame_aircrafts, all_codes, 'Crafts', 'FullName', skip_list, icao_aircraft_lst, icaos_aircraft, iatas_aircraft)
matches_aircrafts = matches_aircrafts[['ICAO', 'IATA', 'researched_value', 'cutoff','orig_database']]
matches_aircrafts = matches_aircrafts.rename(columns={'ICAO':'Aircraft ICAO', 'IATA':'Aircraft IATA',
                                                      'researched_value':'researched_aircraft_value',
                                                      'cutoff':'aircraft_cutoff', 'orig_database':'aircrafts_database'})

### 8. Adding the info to the matched ICAO and IATA designators

In [ ]:
matches_full = add_info(matches_aircrafts, all_codes)

### 9. Creation of final report

If it is needed to unite information about airlines for visual understanding, set parameter **index_visualize** to True in **create_report()** function below

In [ ]:
create_report(overall, matches_full, overall_exp, corrupted+corrupted_, index_visualize=False)

In [ ]:
print(f'Total running time: {(datetime.now()-script_started)}')

----